<a href="https://colab.research.google.com/github/pparkitny/basketball-projection-analysis-AI/blob/main/Analiza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [33]:
import json
data = []

with open('/content/drive/MyDrive/Y_OP/VID_20201110_170806-yolo.json') as json_file:
  data = json.load(json_file)

In [53]:
import pandas as pd

df_rows = list()
for frame_id, frame in enumerate(data):
    for obj in frame:
        df_rows.append([frame_id, obj['name'], obj['percentage_probability'], *obj['box_points']])

df = pd.DataFrame(df_rows, columns=["frame", "object", "proba", "x1", "y1", "x2", "y2"])
print(df)

all_frames = df["frame"].max() # ilość wszystkich klatek
print(all_frames)
all= df['object'].value_counts()
print(all) # drukuje wszystkie

# stworzenie dataframe z rim, gdzie jest ponad 90 % 
rim = df[(df['object']) == 'rim']
gproba_rim = rim[rim['proba'] > 90]
 # stworzenie dataframe z ball, gdzie jest ponad 60 % 
ball = df[(df['object']) == 'ball']
gproba_ball = ball[ball['proba'] > 60]
# stworzenie dataframe z people, gdzie jest ponad 80 % 
people = df[(df['object']) =='people']
gproba_pp= people[people['proba'] > 80]
# połączenie trzech dataframe w jedno
df=pd.concat([gproba_pp, gproba_ball,gproba_rim])

all_good = df['object'].value_counts()
print(all_good) # drukuje prawidłowe

     frame  object      proba    x1   y1    x2   y2
0        0     rim  99.824744  1344  238  1484  344
1        1     rim  99.820614  1343  238  1485  344
2        1    ball  50.594467   280  704   335  737
3        2     rim  99.814391  1344  238  1485  344
4        3     rim  99.811375  1344  238  1485  344
..     ...     ...        ...   ...  ...   ...  ...
232     98     rim  99.751657  1341  235  1485  347
233     98    ball  94.189703   953  293  1009  343
234     99     rim  99.758506  1340  236  1487  346
235     99    ball  89.972377   936  317   994  369
236     99  people  61.721522   249  507   379  882

[237 rows x 7 columns]
99
rim       100
ball       75
people     62
Name: object, dtype: int64
rim       100
ball       67
people     35
Name: object, dtype: int64


In [52]:
# sprawdzanie ile klatek jest poprawnych (3 elementy)
count_frames = df["frame"].max()
n = 0
is_good = 0 # zmienna przechwująca dobre klatki
bad_frames = [] # lista przechowująca złe klatki (potrzebne do openpose)
good_frames = [] # lista przechowująca dobre klatki
while n < count_frames: 
  one_frame = df[(df['frame']) == n]
  x = one_frame['object'].count() # zliczanie ile mamy obiektów w  danej klatce
  if "rim" in one_frame.values and "people" in one_frame.values and 'ball' in one_frame.values and x == 3:
    print("Klatka",n,"jest git")
    is_good += 1
    good_frames.append(n)
  else:
    print("Klatka",n,"nie jest git")
    df = df[df.frame != n]
    bad_frames.append(n)
  n += 1
print("Dobrych klatek:", is_good)
ratio = is_good/count_frames  # jaki współczynnik klatek jest git
print("wspolczynnik dobrych:", ratio)
if ratio > 0.5 :
  print("Spoko film mordo")
else:
  print("Daj inny film byczku")

Klatka 0 nie jest git
Klatka 1 nie jest git
Klatka 2 nie jest git
Klatka 3 nie jest git
Klatka 4 nie jest git
Klatka 5 nie jest git
Klatka 6 nie jest git
Klatka 7 nie jest git
Klatka 8 nie jest git
Klatka 9 nie jest git
Klatka 10 nie jest git
Klatka 11 nie jest git
Klatka 12 nie jest git
Klatka 13 nie jest git
Klatka 14 nie jest git
Klatka 15 nie jest git
Klatka 16 nie jest git
Klatka 17 nie jest git
Klatka 18 nie jest git
Klatka 19 nie jest git
Klatka 20 nie jest git
Klatka 21 nie jest git
Klatka 22 nie jest git
Klatka 23 nie jest git
Klatka 24 nie jest git
Klatka 25 nie jest git
Klatka 26 nie jest git
Klatka 27 nie jest git
Klatka 28 nie jest git
Klatka 29 nie jest git
Klatka 30 nie jest git
Klatka 31 nie jest git
Klatka 32 nie jest git
Klatka 33 nie jest git
Klatka 34 nie jest git
Klatka 35 nie jest git
Klatka 36 nie jest git
Klatka 37 nie jest git
Klatka 38 nie jest git
Klatka 39 nie jest git
Klatka 40 nie jest git
Klatka 41 nie jest git
Klatka 42 nie jest git
Klatka 43 nie jest gi

In [54]:
# i teraz trzeba sprawdzić na ilu klatkach widać przechodzącą pilke przez obrecz (trafiony)
for n in good_frames:
  one_frame= df[(df['frame']) == n]
# wyciągamy koordynaty piłki
  ball_cords = one_frame[(one_frame['object']) == 'ball']
  b_x1 = ball_cords['x1'].item()
  b_x2 = ball_cords['x2'].item()
  b_y1 = ball_cords['y1'].item()
  b_y2 = ball_cords['y2'].item()
# wyliczamy środek piłki
  b_x = (b_x1 + b_x2)/2
  b_y = (b_y1 + b_y2)/2
# wyciągamy koordynaty ramki
  rim_cords = one_frame[(one_frame['object']) == 'rim']
  r_x1 = rim_cords['x1'].item()
  r_x2 = rim_cords['x2'].item()
  r_y1 = rim_cords['y1'].item()
  r_y2 = rim_cords['y2'].item()
# sprawdzamy czy środek piłki przeszedł przez ramkę
  if b_x1 == r_x1 and b_x2 == r_x2 and b_y1 == r_y1 and b_y2 == r_y2:
    df = df[df.frame != n]
  elif r_x1 < b_x and r_x2 > b_x and r_y1 < b_y and r_y2 > b_y:
    print(n, "TRAFIONE!")
  else:
    print(n, "nie trafione :(")

44 nie trafione :(
45 nie trafione :(
49 nie trafione :(
50 nie trafione :(
56 nie trafione :(
57 nie trafione :(
58 nie trafione :(
59 nie trafione :(
60 nie trafione :(
61 nie trafione :(
62 nie trafione :(
63 nie trafione :(
64 nie trafione :(
65 nie trafione :(
66 nie trafione :(
67 nie trafione :(
76 nie trafione :(
89 nie trafione :(
90 nie trafione :(
91 nie trafione :(
94 nie trafione :(
96 nie trafione :(


In [42]:
#pobranie dataframe openpose
with open('/content/drive/MyDrive/Y_OP/VID_20201110_170806-openpose.json') as json_file:
  data_op = json.load(json_file)

df_rows = list()
for frame_id, frame in enumerate(data_op):
  for person_id, person in enumerate(frame['persons']):
    for part_id, part in enumerate(person):
      df_rows.append([frame_id, person_id, part_id, *part])

df_op = pd.DataFrame(df_rows, columns=["frame", "person", "part", "x", "y", "proba"])

# usuwanie klatek w których YOLO nie wykryło 3 dobrych obiektów 
#   (tymczasowo usunięte bo chyba lepiej wykrywać czy punkt z op znajduje się w kwadracie z YOLO na każdej klatce)
# for i in bad_frames:
#   df_op = df_op[df_op.frame != i]
# print(df_op)

# zobaczenie czy środek ciała w ogóle jest na klatce xD
# test = df_op[df_op['part'] == 8]
# print(test)

# usuwanie niepotrzebych punktów
points = [15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
for i in points:
  df_op = df_op[df_op.part != i]
# print(df_op)

# sprawdzenie wiekszego prawdopodobieństwa i usunięcie osób jeśli jest więcej niż jedna
for frame_id, df_frame in df_op.groupby("frame"):
  probas = []
  for person_id, df_person in df_frame.groupby("person"):
    probas.append(df_person['proba'].mean())
  person_correct_id = probas.index(max(probas))
  df_op = df_op[(df_op["frame"] != frame_id) | ((df_op["frame"] == frame_id) & (df_op["person"] == person_correct_id))]

# jaka osoba została na klatce
#for frame_id, df_frame in df_op.groupby("frame"):
#   print(df_frame["person"].unique())

print(df_op)

      frame  person  part        x        y     proba
0         0       0     0  288.889  597.848  0.917221
1         0       0     1  230.078  627.393  0.781033
2         0       0     2  233.060  633.253  0.829421
3         0       0     3  244.721  709.805  0.898068
4         0       0     4  294.804  739.328  0.938137
...     ...     ...   ...      ...      ...       ...
4560     99       0    10  309.497  862.958  0.813158
4561     99       0    11  268.279  980.631  0.844107
4562     99       0    12  288.991  759.839  0.834543
4563     99       0    13  318.486  856.831  0.787023
4564     99       0    14  277.109  957.111  0.719136

[1500 rows x 6 columns]


In [55]:
# Sprawdzamy czy człowiek jest w człowieku xd
one_frame_yolo = df[(df['frame'].isin(good_frames)) & (df['object'] == 'people')]
one_frame_op = df_op[(df_op['frame'].isin(good_frames)) & (df_op['part'] == 8)]
# merge obu dataframe
person_check = pd.merge(one_frame_yolo, one_frame_op, how='inner', on='frame')
# sprawdzamy czy środek człowieka z OpenPose znajduje się wewnątrz ramki People z Yolo
def person_condition(x1, x2, x, y1, y2, y):
  if x1 < x and x2 > x and y1 < y and y2 > y:
    return True
  return False
  
person_check['good'] = [person_condition(*row) for row in zip(person_check['x1'], person_check['x2'], person_check['x'], person_check['y1'], person_check['y2'], person_check['y'])]
person_ratio = person_check['good'].value_counts().loc[True] / person_check.shape[0]

if person_ratio > 0.90:
  print("W pyte film", person_ratio)
else:
  print("Dej nowy", person_ratio)

W pyte film 1.0
